In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_clean_and_preprocess').getOrCreate()

In [ ]:
df = spark.read.option("sep","\t").csv("column_headers.tsv",header='true', inferSchema='true')
df1 = spark.read.option("sep","\t").csv("hit_data.tsv",header='false', inferSchema='true')

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import functions as sf
sc = SparkContext.getOrCreate()
sqlc = pyspark.SQLContext(sc)

In [ ]:
result = df.union(df1)
#result.head(1)
#result.count()

In [ ]:
NoValuedExcludeHit=result.filter(result.exclude_hit == 0)
#NoValuedExcludeHit.count()

In [ ]:
ValidHitSourceWithout5=NoValuedExcludeHit.filter(NoValuedExcludeHit.hit_source != 5 )
ValidHitSourceWithout7=ValidHitSourceWithout5.filter(ValidHitSourceWithout5.hit_source != 7 )
ValidHitSourceWithout9=ValidHitSourceWithout7.filter(ValidHitSourceWithout7.hit_source != 9 )
ValidHitSourceWithout8=ValidHitSourceWithout9.filter(ValidHitSourceWithout9.hit_source != 8 )
#ValidHitSourceWithout8.count()

In [ ]:
MergedColumns = ValidHitSourceWithout8.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#apart from post_ columns considering column obtained after joining the two post_ columns as mentioned above and mcvisid
MergedColumns=MergedColumns.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
MergedColumns=MergedColumns.withColumnRenamed('mcvisid', 'post_mcvisid')

In [ ]:
#dropping the extraneous columns
DropPostVHL=MergedColumns.drop('post_visid_high','post_visid_low')
#len(DropPostVHL.columns)

In [ ]:
#considering only post_ columns
newDataF= DropPostVHL.select(*filter(lambda col:'post_' in col, DropPostVHL.columns))
#len(newDataF.columns)
#newDataF.count()

In [ ]:
MergedColumns.select('post_referrer')

In [ ]:
MergedColumns=MergedColumns.withColumnRenamed('post_referrer','post_evar_r')

In [ ]:
MergedColumns.select('post_evar_r')

In [ ]:
#filter eVar to find null values
eVar=newDataF.select(*filter(lambda col:'post_evar' in col, newDataF.columns))

In [ ]:
NoNullevar=newDataF.select( "post_evar1", "post_evar2", "post_evar8", "post_evar11", "post_evar26","post_evar27","post_evar28","post_evar33","post_evar34","post_evar35","post_evar38", "post_evar39", "post_evar40", "post_evar47","post_evar48","post_evar49","post_evar51","post_evar53","post_evar63","post_evar64","post_evar65","post_evar68","post_evar70","post_evar71", "post_evar77","post_evar78","post_evar80","post_evar98","post_evar99","post_evar101","post_evar102",'post_referrer')

In [ ]:
NoNullevar=NoNullevar.select( "post_evar27", "post_evar33", "post_evar34", "post_evar35", "post_evar39","post_evar47","post_evar71","post_evar78",'post_referrer')

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
MergedColumns = NoNullevar.withColumn('joined_column', sf.concat(sf.col('post_evar47'),sf.lit(':'), sf.col('post_evar34')))

In [ ]:
#Converting string data to float
indexerpost_evar34and47 = StringIndexer(inputCol='joined_column', outputCol='post_evar34post47_cat')
indexed = indexerpost_evar34and47.fit(MergedColumns).transform(MergedColumns)
#indexed.head(5)

In [ ]:
indexedwithoutpost_evar34and47=indexed.drop('post_evar34','post_evar47')

In [ ]:
#Converting string data to float
indexerpost_evar35 = StringIndexer(inputCol='post_evar35', outputCol='post_evar35_cat')
indexed = indexerpost_evar35.fit(indexedwithoutpost_evar34and47).transform(indexedwithoutpost_evar34and47)
#indexed.head(5)

In [ ]:
indexedwithoutpost_evar35=indexed.drop('post_evar35')

In [ ]:
#Converting string data to float
indexerpost_evar27 = StringIndexer(inputCol='post_evar27', outputCol='post_evar27_cat')
indexed = indexerpost_evar27.fit(indexedwithoutpost_evar35).transform(indexedwithoutpost_evar35)

In [ ]:
indexedwithoutpost_evar27=indexed.drop('post_evar27')

In [ ]:
#selecting a subset
sub=indexedwithoutpost_evar27.select('post_evar34post47_cat','post_evar35_cat','post_referrer','post_evar27_cat')
#indexed.select("post_referrer").show(20,False)

In [ ]:
#selecting subset
SelectedRows=sub.filter(sub.post_referrer.endswith('/cart')).select('post_evar34post47_cat','post_evar35_cat','post_evar27_cat','post_referrer')
#SelectedRows.count()

In [ ]:
#finding correlation 
from pyspark.sql.functions import corr
SelectedRows.select(corr('post_evar34post47_cat', 'post_evar35_cat')).show()

In [ ]:
#finding variance for post_evar27
SelectedRows.stat.cov('post_evar27_cat','post_evar27_cat')